### split to folder

In [2]:
import os,json,csv,shutil

trainRoot = "/home/edward/test/OUTtest/CMU_AICenter/NIH_CXR_Images/train/"
testRoot = "/home/edward/test/OUTtest/CMU_AICenter/NIH_CXR_Images/test/"
trainCsv = '/home/edward/test/OUTtest/CMU_AICenter/NIH_CXR_train.csv'
testCsv = '/home/edward/test/OUTtest/CMU_AICenter/NIH_CXR_test.csv'

trainLabelDict = {}
testLabelDict = {}

with open(trainCsv) as f:
    rows = csv.reader(f)
    for row in rows:
#         print(row)
        if('png' in row[0]):
            trainLabelDict[row[0]]=row[1]
# print(trainLabelDict)

with open(testCsv) as f:
    rows = csv.reader(f)
    for row in rows:
#         print(row)
        if('png' in row[0]):
            testLabelDict[row[0]]=row[1]
# print(testLabelDict)

for fp,dl,fl in os.walk(trainRoot):
    # print(fp,dl,fl)
    if(len(fl)>0):
        for i in fl:
            if(i.split('.')[-1]=='png'):
                # print(fp + i)
                if(trainLabelDict[i] == 'Atelectasis'):
#                     label = 1
                    if not(os.path.isdir('./Split_Folder/train/Atelectasis')):
                        os.makedirs('./Split_Folder/train/Atelectasis')
                    shutil.copy(fp + '/' + i, './Split_Folder/train/Atelectasis/' + i)
                if(trainLabelDict[i] == 'No Finding'):
#                     label = 0
                    if not(os.path.isdir('./Split_Folder/train/No Finding')):
                        os.makedirs('./Split_Folder/train/No Finding')
                    shutil.copy(fp + '/' + i, './Split_Folder/train/No Finding/' + i)
                # print(fp + '/' + i,label)
                # with open(trainList,'a') as f:
                #     f.writelines(fp + '/' + i + ',' + str(label) + '\n')
#                 fileDict['train'].append([fp + i,label])

for fp,dl,fl in os.walk(testRoot):
    # print(fp,dl,fl)
    if(len(fl)>0):
        for i in fl:
            if(i.split('.')[-1]=='png'):
                # print(fp + i)
                if(testLabelDict[i] == 'Atelectasis'):
#                     label = 1
                    if not(os.path.isdir('./Split_Folder/test/Atelectasis')):
                        os.makedirs('./Split_Folder/test/Atelectasis')
                    shutil.copy(fp + '/' + i, './Split_Folder/test/Atelectasis/' + i)
                if(testLabelDict[i] == 'No Finding'):
#                     label = 0
                    if not(os.path.isdir('./Split_Folder/test/No Finding')):
                        os.makedirs('./Split_Folder/test/No Finding')
                    shutil.copy(fp + '/' + i, './Split_Folder/test/No Finding/' + i)
                # print(fp + '/' + i,label)
                # with open(trainList,'a') as f:
                #     f.writelines(fp + '/' + i + ',' + str(label) + '\n')
#                 fileDict['test'].append([fp + i,label])

## Genrate Train List

In [7]:
import os,json,csv

trainRoot = "/home/edward/test/OUTtest/CMU_AICenter/Split_Folder/train/"
testRoot = "/home/edward/test/OUTtest/CMU_AICenter/Split_Folder/test/"
trainList='/home/edward/test/OUTtest/CMU_AICenter/trainReal1.json'
fileDict ={
    'train' : [],
    'test' : []
    }

tra=0
trn=0
tsa=0
tsn=0
for fp,dl,fl in os.walk(trainRoot):
    # print(fp,dl,fl)
    if(len(fl)>0):
        for i in fl:
            if(i.split('.')[-1]=='png'):
                # print(fp + i)
                if(fp.split(trainRoot)[-1].split('/')[0] == 'Atelectasis'):
                    label = 1
                    tra += 1
                if(fp.split(trainRoot)[-1].split('/')[0] == 'No Finding'):
                    label = 0
                    trn += 1
                # print(fp + '/' + i,label)
                # with open(trainList,'a') as f:
                #     f.writelines(fp + '/' + i + ',' + str(label) + '\n')
                fileDict['train'].append([fp + '/' + i,label])

for fp,dl,fl in os.walk(testRoot):
    # print(fp,dl,fl)
    if(len(fl)>0):
        for i in fl:
            if(i.split('.')[-1]=='png'):
                # print(fp + i)
                if(fp.split(testRoot)[-1].split('/')[0] == 'Atelectasis'):
                    label = 1
                    tsa += 1
                if(fp.split(testRoot)[-1].split('/')[0] == 'No Finding'):
                    label = 0
                    tsn += 1
                # print(fp + '/' + i,label)
                # with open(trainList,'a') as f:
                #     f.writelines(fp + '/' + i + ',' + str(label) + '\n')
                fileDict['test'].append([fp + '/' + i,label])

print(tra,trn,tsa,tsn)

with open(trainList,'w') as f:
    json.dump(fileDict,f,indent=4)


3307 3993 27 270


### Undersampling "No Finding" 

In [ ]:
import os,shutil,random

oldNFPath = "/home/edward/test/OUTtest/CMU_AICenter/Split_Folder/train_no/"
newNFPath = "/home/edward/test/OUTtest/CMU_AICenter/Split_Folder/train/No Finding/"

flist = os.listdir(oldNFPath)
# print(flist)
# print(80*'x')
random.shuffle(flist)
# print(flist)
sample = 251
for i in range(sample):
    shutil.move(oldNFPath + flist[i], newNFPath + flist[i])

## Train

In [1]:
from tensorflow.keras import Model
from tensorflow.keras.regularizers import l2
from tensorflow.keras.layers import  Conv2D, BatchNormalization, Activation, Input, AveragePooling2D, Dense, Dropout, Flatten
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model
# from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from keras.utils import to_categorical
# from tensorflow.keras.layers import Input
from keras import backend as K
import os, datetime, tensorflow as tf, numpy as np,cv2,json


from skimage.io import imread
from skimage.transform import resize

In [2]:
freq_pos = [3307/6700]
freq_neg = [3993/6700]
print(freq_pos)
print(freq_neg)

[0.4935820895522388]
[0.5959701492537314]


In [3]:
weights_pos = K.constant(freq_neg,dtype='float32')
weights_neg = K.constant(freq_pos,dtype='float32')
epsilon=1e-7
def weighted_loss(y_true, y_pred):
    loss = 0.0
    loss_pos = -1 * K.sum( weights_pos * y_true * K.log(y_pred + epsilon), axis=-1)
    loss_neg = -1 * K.sum( weights_neg * (1 - y_true) * K.log(1 - y_pred + epsilon) ,axis=-1)
    return (loss_pos+loss_neg)/2

def binary_focal_loss(gamma=2, alpha=0.25):
    alpha = tf.constant(alpha, dtype=tf.float32)
    gamma = tf.constant(gamma, dtype=tf.float32)

    def binary_focal_loss_fixed(y_true, y_pred):
        y_true = tf.cast(y_true, tf.float32)
        alpha_t = y_true*alpha + (K.ones_like(y_true)-y_true)*(1-alpha)
    
        p_t = y_true*y_pred + (K.ones_like(y_true)-y_true)*(K.ones_like(y_true)-y_pred) + K.epsilon()
        focal_loss = - alpha_t * K.pow((K.ones_like(y_true)-p_t),gamma) * K.log(p_t)
        return K.mean(focal_loss)
    return binary_focal_loss_fixed

In [4]:
def resnet_layer(inputs,
                 num_filters=16,
                 kernel_size=3,
                 strides=1,
                 activation='relu',
                 batch_normalization=True,
                 conv_first=True):
    
    conv = Conv2D(num_filters,
                  kernel_size=kernel_size,
                  strides=strides,
                  padding='same',
                  kernel_initializer='he_normal',
                  kernel_regularizer=l2(1e-4))

    x = inputs
    if conv_first:
        x = conv(x)
        if batch_normalization:
            x = BatchNormalization()(x)
        if activation is not None:
            x = Activation(activation)(x)
    else:
        if batch_normalization:
            x = BatchNormalization()(x)
        if activation is not None:
            x = Activation(activation)(x)
        x = conv(x)
    return x

In [5]:
def resnet_v2(input_shape, depth,num_classes):
    if (depth - 2) % 9 != 0:
        raise ValueError('depth should be 9n+2 (eg 56 or 110 in [b])')
    # Start model definition.
    num_filters_in = 16
    num_res_blocks = int((depth - 2) / 9)

    inputs = Input(shape=input_shape)
    # v2 performs Conv2D with BN-ReLU on input before splitting into 2 paths
    x = resnet_layer(inputs=inputs,
                     num_filters=num_filters_in,
                     conv_first=True)

    # Instantiate the stack of residual units
    for stage in range(3):
        for res_block in range(num_res_blocks):
            activation = 'relu'
            batch_normalization = True
            strides = 1
            if stage == 0:
                num_filters_out = num_filters_in * 4
                if res_block == 0:  # first layer and first stage
                    activation = None
                    batch_normalization = False
            else:
                num_filters_out = num_filters_in * 2
                if res_block == 0:  # first layer but not first stage
                    strides = 2    # downsample

            # bottleneck residual unit
            y = resnet_layer(inputs=x,
                             num_filters=num_filters_in,
                             kernel_size=1,
                             strides=strides,
                             activation=activation,
                             batch_normalization=batch_normalization,
                             conv_first=False)
            y = resnet_layer(inputs=y,
                             num_filters=num_filters_in,
                             conv_first=False)
            y = resnet_layer(inputs=y,
                             num_filters=num_filters_out,
                             kernel_size=1,
                             conv_first=False)
            if res_block == 0:
                # linear projection residual shortcut connection to match
                # changed dims
                x = resnet_layer(inputs=x,
                                 num_filters=num_filters_out,
                                 kernel_size=1,
                                 strides=strides,
                                 activation=None,
                                 batch_normalization=False)
            x = tf.keras.layers.add([x, y])

        num_filters_in = num_filters_out

    # Add classifier on top.
    # v2 has BN-ReLU before Pooling
    x = BatchNormalization()(x)
    x = Activation('relu',name='last_activation_layer')(x)
    x = AveragePooling2D(pool_size=8)(x)
    y = Flatten()(x)
    outputs = Dense(num_classes,
                    activation='sigmoid',
                    kernel_initializer='he_normal')(y)

    # Instantiate model.
    model = Model(inputs=inputs, outputs=outputs)
    return model

In [6]:

# from tensorflow.python.client import device_lib
# print(device_lib.list_local_devices())
# print(tf.__version__)
# print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
# tf.test.is_gpu_available()



class datagenator(tf.keras.utils.Sequence):

    'Generates data for Keras'
    def __init__(self, list_IDs, labels, batch_size=10, shuffle=True,n_classes = 2,mode='train'):
        'Initialization'
        self.batch_size = batch_size
        self.labels = labels
        self.list_IDs = list_IDs
        self.shuffle = shuffle
        self.n_classes = n_classes
        self.mode = mode
        self.err={
            'effective':[],
            'invalid':[]
        }
        self.on_epoch_end()

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.list_IDs) / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

        # Find list of IDs
        list_IDs_temp = [self.list_IDs[k] for k in indexes]

        # Generate data
        X, y = self.__data_generation(list_IDs_temp)
        # print(80*'!')
        # print(np.shape(X),np.shape(y))

        return X, y

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.list_IDs))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

    def __data_generation(self, list_IDs_temp):
        'Generates data containing batch_size samples' # X : (n_samples, *dim, n_channels)
        # Initialization
        X = [] #np.empty((self.batch_size, *self.dim, self.n_channels))
        y = [] #np.empty((self.batch_size), dtype=int)

        # Generate data
        for ID in list_IDs_temp:
            # Store sample
#             print(ID)
            img = cv2.imread(ID)
            in_ = np.array(img, dtype=np.float32)
            shape = np.shape(in_)
#             print(shape)
#             try:
            side = 128
            if(shape[0]>shape[1]):
                    in_=cv2.resize(in_,(side,shape[1]*side/shape[0]),interpolation=cv2.INTER_CUBIC)
                    newshape=np.shape(in_)
                    addsp=(side-newshape[0])/2
                    fr_sp=np.zeros((addsp,side,3),np.float32)
                    if((side-newshape[0])%2!=0):
                            addsp+=1
                    bk_sp=np.zeros((addsp,side,3),np.float32)
                    in_=np.concatenate((fr_sp,in_,bk_sp),axis=0)
            elif(shape[0]<shape[1]):
                    in_=cv2.resize(in_,(shape[0]*side/shape[1],side),interpolation=cv2.INTER_CUBIC)
                    newshape=np.shape(in_)
                    addsp=(side-newshape[1])/2
                    fr_sp=np.zeros((side,addsp,3),np.float32)
                    if((side-newshape[1])%2!=0):
                            addsp+=1
                    bk_sp=np.zeros((side,addsp,3),np.float32)
                    in_=np.concatenate((fr_sp,in_,bk_sp),axis=1)

            else:
                    in_=cv2.resize(in_,(side,side),interpolation=cv2.INTER_CUBIC)
#             except:
#                 in_ = np.array(np.random.randint(0,256,size = (517,517,3)), dtype=np.float32)
#                 # print(ID)
#                 if('effective' in ID):
#                     if not(ID.split('effective')[-1].split('/')[1] in self.err['effective']):
#                         self.err['effective'].append(ID.split('effective')[-1].split('/')[1])
#                         self.err['effective'].sort()
#                 if('invalid' in ID):
#                     if not(ID.split('invalid')[-1].split('/')[1] in self.err['invalid']):
#                         self.err['invalid'].append(ID.split('invalid')[-1].split('/')[1])
#                         self.err['invalid'].sort()

#                 if(self.mode =='train'):
#                     with open('err_train.json','w') as f:
#                         # f.writelines(ID + '\n')
#                         json.dump(self.err,f,indent=4)
                
#                 if(self.mode =='val'):
#                     with open('err_test.json','w') as f:
#                         # f.writelines(ID + '\n')
#                         json.dump(self.err,f,indent=4)
            
            X.append(in_)

            # Store class
            y.append(self.labels[ID])

        return np.array(X), np.array(to_categorical(y, num_classes=self.n_classes))


imgList = json.load(open('/home/edward/test/OUTtest/CMU_AICenter/trainReal1.json'))

partition = {
    'train' :[],
    'test' :[]
}
labels = {}

for i in imgList['train']:
    partition['train'].append(i[0])
    labels[i[0]] = i[1]
for i in imgList['test']:
    partition['test'].append(i[0])
    labels[i[0]] = i[1]

# this could also be the output a different Keras model or layer
# input_tensor = Input(shape=(517, 517, 3))
# # create the base pre-trained model
# base_model = InceptionV3(   
#     input_tensor=input_tensor,
#     weights=None,
#     include_top=True,
#     classes=2,
# ) #input_tensor=input_tensor, weights='imagenet', 

# add a global spatial average pooling layer
# x = base_model.output
# x = GlobalAveragePooling2D()(x)
# let's add a fully-connected layer
# x = Dense(1024, activation='relu')(x)
# and a logistic layer -- let's say we have 200 classes
# predictions = Dense(2, activation='softmax')(x)

# this is the model we will train
model = resnet_v2((128,128,3),(9*6)+2,2)#Model(inputs=base_model.input, outputs=x) resnet_v2((128,128,3),(9*6)+2,2)
# model.summary()

# compile the model (should be done *after* setting layers to non-trainable)
model.compile(optimizer='nadam', loss='categorical_crossentropy', metrics=['accuracy', 'Precision', 'Recall', 'AUC']) 
#'categorical_crossentropy' [binary_focal_loss(alpha=.25, gamma=2)] weighted_loss

# model.load_weights("/home/edward/test/OUTtest/CMU_AICenter/RMS_focal/weights.30-0.96.hdf5", by_name=True)

logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)
checkPointPath = './checkpoint/weights.{epoch:02d}-{auc:.2f}.hdf5'
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(checkPointPath, monitor='auc', save_weights_only=True)

training_generator = datagenator(partition['train'], labels, batch_size=50, n_classes = 2, shuffle=True, mode='train')
val_generator = datagenator(partition['test'], labels, batch_size=9, n_classes = 2, shuffle=True,mode='val')
# x_train = np.random.randint(0,256,size=(1000,517,517,3))
# print(np.shape(x_train))
# y_train=to_categorical(np.random.randint(0,2,size=(1000)))
# print('y_train= ',y_train)
# print([[0.,1.]])

model.fit(
    x = training_generator, 
    epochs = 100,
    callbacks = [tensorboard_callback, checkpoint_callback],
    validation_data = val_generator
    ) #validation_data=(x_test, y_test),batch_size = 10, 

log_writer = tf.summary.create_file_writer(logdir)

model.save_weights(os.path.join("weights", datetime.datetime.now().strftime("%Y%m%d-%H%M%S") + '.h5'))

Epoch 1/100
  1/146 [..............................] - ETA: 0s - loss: 1.9224 - accuracy: 0.4600 - precision: 0.4600 - recall: 0.4600 - auc: 0.4156WARNING:tensorflow:From /home/edward/.local/lib/python3.6/site-packages/tensorflow/python/ops/summary_ops_v2.py:1277: stop (from tensorflow.python.eager.profiler) is deprecated and will be removed after 2020-07-01.
Instructions for updating:
use `tf.profiler.experimental.stop` instead.
146/146 [==============================] - 93s 638ms/step - loss: 1.4171 - accuracy: 0.7793 - precision: 0.7311 - recall: 0.7885 - auc: 0.8180 - val_loss: 3.3549 - val_accuracy: 0.1145 - val_precision: 0.1261 - val_recall: 0.0505 - val_auc: 0.1160
Epoch 2/100
146/146 [==============================] - 92s 628ms/step - loss: 1.0308 - accuracy: 0.8314 - precision: 0.7957 - recall: 0.7837 - auc: 0.8637 - val_loss: 2.9177 - val_accuracy: 0.1010 - val_precision: 0.1635 - val_recall: 0.0572 - val_auc: 0.1149
Epoch 3/100
146/146 [==============================] - 91s

KeyboardInterrupt: 

## Test

In [1]:
from keras.utils import to_categorical
# from tensorflowf.keras.utils import Sequence
from tensorflow.keras.layers import Input
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.models import Model
from keras import backend as K
import numpy as np,cv2,tensorflow as tf,json,os,datetime
from tensorflow.keras.regularizers import l2
from tensorflow.keras.layers import  Conv2D, BatchNormalization, Activation, Input, AveragePooling2D, Dense, Dropout, Flatten
from tensorflow.keras.preprocessing import image


from skimage.io import imread
from skimage.transform import resize

In [2]:
weights_pos = K.constant([3993/4242],dtype='float32')
weights_neg = K.constant([249/4242],dtype='float32')
epsilon=1e-7
def weighted_loss(y_true, y_pred):
    loss = 0.0
    loss_pos = -1 * K.sum( weights_pos * y_true * K.log(y_pred + epsilon), axis=-1)
    loss_neg = -1 * K.sum( weights_neg * (1 - y_true) * K.log(1 - y_pred + epsilon) ,axis=-1)
    return (loss_pos+loss_neg)/2

def binary_focal_loss(gamma=2, alpha=0.25):
    alpha = tf.constant(alpha, dtype=tf.float32)
    gamma = tf.constant(gamma, dtype=tf.float32)

    def binary_focal_loss_fixed(y_true, y_pred):
        y_true = tf.cast(y_true, tf.float32)
        alpha_t = y_true*alpha + (K.ones_like(y_true)-y_true)*(1-alpha)
    
        p_t = y_true*y_pred + (K.ones_like(y_true)-y_true)*(K.ones_like(y_true)-y_pred) + K.epsilon()
        focal_loss = - alpha_t * K.pow((K.ones_like(y_true)-p_t),gamma) * K.log(p_t)
        return K.mean(focal_loss)
    return binary_focal_loss_fixed

In [3]:
def resnet_layer(inputs,
                 num_filters=16,
                 kernel_size=3,
                 strides=1,
                 activation='relu',
                 batch_normalization=True,
                 conv_first=True):
    
    conv = Conv2D(num_filters,
                  kernel_size=kernel_size,
                  strides=strides,
                  padding='same',
                  kernel_initializer='he_normal',
                  kernel_regularizer=l2(1e-4))

    x = inputs
    if conv_first:
        x = conv(x)
        if batch_normalization:
            x = BatchNormalization()(x)
        if activation is not None:
            x = Activation(activation)(x)
    else:
        if batch_normalization:
            x = BatchNormalization()(x)
        if activation is not None:
            x = Activation(activation)(x)
        x = conv(x)
    return x

In [4]:
def resnet_v2(input_shape, depth,num_classes):
    if (depth - 2) % 9 != 0:
        raise ValueError('depth should be 9n+2 (eg 56 or 110 in [b])')
    # Start model definition.
    num_filters_in = 16
    num_res_blocks = int((depth - 2) / 9)

    inputs = Input(shape=input_shape)
    # v2 performs Conv2D with BN-ReLU on input before splitting into 2 paths
    x = resnet_layer(inputs=inputs,
                     num_filters=num_filters_in,
                     conv_first=True)

    # Instantiate the stack of residual units
    for stage in range(3):
        for res_block in range(num_res_blocks):
            activation = 'relu'
            batch_normalization = True
            strides = 1
            if stage == 0:
                num_filters_out = num_filters_in * 4
                if res_block == 0:  # first layer and first stage
                    activation = None
                    batch_normalization = False
            else:
                num_filters_out = num_filters_in * 2
                if res_block == 0:  # first layer but not first stage
                    strides = 2    # downsample

            # bottleneck residual unit
            y = resnet_layer(inputs=x,
                             num_filters=num_filters_in,
                             kernel_size=1,
                             strides=strides,
                             activation=activation,
                             batch_normalization=batch_normalization,
                             conv_first=False)
            y = resnet_layer(inputs=y,
                             num_filters=num_filters_in,
                             conv_first=False)
            y = resnet_layer(inputs=y,
                             num_filters=num_filters_out,
                             kernel_size=1,
                             conv_first=False)
            if res_block == 0:
                # linear projection residual shortcut connection to match
                # changed dims
                x = resnet_layer(inputs=x,
                                 num_filters=num_filters_out,
                                 kernel_size=1,
                                 strides=strides,
                                 activation=None,
                                 batch_normalization=False)
            x = tf.keras.layers.add([x, y])

        num_filters_in = num_filters_out

    # Add classifier on top.
    # v2 has BN-ReLU before Pooling
    x = BatchNormalization()(x)
    x = Activation('relu',name='last_activation_layer')(x)
    x = AveragePooling2D(pool_size=8)(x)
    y = Flatten()(x)
    outputs = Dense(num_classes,
                    activation='sigmoid',
                    kernel_initializer='he_normal')(y)

    # Instantiate model.
    model = Model(inputs=inputs, outputs=outputs)
    return model

In [5]:
class datagenator(tf.keras.utils.Sequence):

    'Generates data for Keras'
    def __init__(self, list_IDs, labels, batch_size=10, shuffle=True,n_classes = 10,mode='train'):
        'Initialization'
        self.batch_size = batch_size
        self.labels = labels
        self.list_IDs = list_IDs
        self.shuffle = shuffle
        self.n_classes = n_classes
        self.mode = mode
        self.err={
            'effective':[],
            'invalid':[]
        }
        self.on_epoch_end()
        
#         print(len(self.list_IDs))

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.list_IDs) / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

        # Find list of IDs
        list_IDs_temp = [self.list_IDs[k] for k in indexes]

        # Generate data
        X, y = self.__data_generation(list_IDs_temp)
        # print(80*'!')
        # print(np.shape(X),np.shape(y))

        return X, y

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.list_IDs))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

    def __data_generation(self, list_IDs_temp):
        'Generates data containing batch_size samples' # X : (n_samples, *dim, n_channels)
        # Initialization
        X = [] #np.empty((self.batch_size, *self.dim, self.n_channels))
        y = [] #np.empty((self.batch_size), dtype=int)

        # Generate data
        for ID in list_IDs_temp:
            # Store sample
            img = cv2.imread(ID)
            in_ = np.array(img, dtype=np.float32)
            shape=np.shape(in_)
            try:
                side = 128
                if(shape[0]>shape[1]):
                        in_=cv2.resize(in_,(side,shape[1]*side/shape[0]),interpolation=cv2.INTER_CUBIC)
                        newshape=np.shape(in_)
                        addsp=(side-newshape[0])/2
                        fr_sp=np.zeros((addsp,side,3),np.float32)
                        if((side-newshape[0])%2!=0):
                                addsp+=1
                        bk_sp=np.zeros((addsp,side,3),np.float32)
                        in_=np.concatenate((fr_sp,in_,bk_sp),axis=0)
                elif(shape[0]<shape[1]):
                        in_=cv2.resize(in_,(shape[0]*side/shape[1],side),interpolation=cv2.INTER_CUBIC)
                        newshape=np.shape(in_)
                        addsp=(side-newshape[1])/2
                        fr_sp=np.zeros((side,addsp,3),np.float32)
                        if((side-newshape[1])%2!=0):
                                addsp+=1
                        bk_sp=np.zeros((side,addsp,3),np.float32)
                        in_=np.concatenate((fr_sp,in_,bk_sp),axis=1)

                else:
                        in_=cv2.resize(in_,(side,side),interpolation=cv2.INTER_CUBIC)
            except:
                in_ = np.array(np.random.randint(0,256,size = (517,517,3)), dtype=np.float32)
                # print(ID)
                if('effective' in ID):
                    if not(ID.split('effective')[-1].split('/')[1] in self.err['effective']):
                        self.err['effective'].append(ID.split('effective')[-1].split('/')[1])
                        self.err['effective'].sort()
                if('invalid' in ID):
                    if not(ID.split('invalid')[-1].split('/')[1] in self.err['invalid']):
                        self.err['invalid'].append(ID.split('invalid')[-1].split('/')[1])
                        self.err['invalid'].sort()

                if(self.mode =='train'):
                    with open('err_train.json','w') as f:
                        # f.writelines(ID + '\n')
                        json.dump(self.err,f,indent=4)
                
                if(self.mode =='val'):
                    with open('err_test.json','w') as f:
                        # f.writelines(ID + '\n')
                        json.dump(self.err,f,indent=4)
            
            X.append(in_)

            # Store class
            y.append(self.labels[ID])

        return np.array(X), np.array(to_categorical(y, num_classes=self.n_classes))

imgList = json.load(open('/home/edward/test/OUTtest/CMU_AICenter/trainReal1.json'))

partition = {
    'train':[],
    'test' :[]
}
labels = {}

# print(len(imgList['test']))
for i in imgList['train']:
    if not('output' in i[0]):
        partition['train'].append(i[0])
        labels[i[0]] = i[1]
for i in imgList['test']:
    partition['test'].append(i[0])
    labels[i[0]] = i[1]

train__generator = datagenator(partition['train'], labels, batch_size=6, n_classes = 2, shuffle=False, mode='train')
val__generator = datagenator(partition['test'], labels, batch_size=9, n_classes = 2, shuffle=False, mode='val')

# input_tensor = Input(shape=(517, 517, 3))
# base_model = InceptionV3(   
#     input_tensor=input_tensor,
#     weights=None,
#     include_top=True,
#     classes=2,
# )
# x = base_model.output

model = resnet_v2((128,128,3),(9*6)+2,2) #Model(inputs=base_model.input, outputs=x)
model.compile(optimizer='nadam', loss='categorical_crossentropy', metrics=['accuracy', 'Precision', 'Recall', 'AUC'])
#'categorical_crossentropy' [binary_focal_loss(alpha=.25, gamma=2)] weighted_loss

# model.load_weights("/home/edward/test/OUTtest/CMU_AICenter/checkpoint/weights.100-0.96.hdf5", by_name=True)

logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)


In [10]:
#==============evaluate======================
evalValue = model.evaluate(
    x=val__generator,
    callbacks=tensorboard_callback,
    return_dict=True,
)
print(evalValue)

4242/4242 [==============================] - 52s 12ms/step - loss: 0.0448 - accuracy: 0.9413 - precision: 0.7775 - recall: 0.9736 - auc: 0.9638
{'loss': 0.04483475536108017, 'accuracy': 0.9413012862205505, 'precision': 0.7774849534034729, 'recall': 0.9735973477363586, 'auc': 0.9637787342071533}


In [6]:
#==============predict=======================
value = model.predict(
    x=val__generator,
    callbacks=tensorboard_callback,
) #use_multiprocessing=True, 

# print(value,np.shape(value))
print(np.shape(value))

predictDict={}

for i in range(len(partition['test'])):
    predictDict[partition['test'][i]]=value[i].tolist()
#     print(partition['test'][i],np.argmax(value[i]))

with open('predict1.json','w') as f:
    json.dump(predictDict,f,indent=4)

(297, 2)


In [ ]:
modelRoot = "/home/edward/test/OUTtest/CMU_AICenter/checkpoint/"
modelList = os.listdir(modelRoot)
modelList.sort()
for modelPath in modelList:
#     print(modelPath,modelPath.split('.')[1].split('-')[0])
    model.load_weights(modelRoot + modelPath, by_name=True)
    value = model.predict(
        x=train__generator,
        callbacks=tensorboard_callback,
    )
    predictDict={}
    for i in range(len(partition['train'])):
        predictDict[partition['train'][i]]=value[i].tolist()
    errCount = [0,0,0,0] #TP,TN,FP,FN
    for k,v in predictDict.items(): 
        pre = np.argmax(v)
        real = labels[k]
#         print(k,real, pre)
    #         if(pre != real):
    #             print(np.argmax(v),labels[k])
        if(real == 1):
            if(pre == 1):
                errCount[0] += 1
            else:
                errCount[3] += 1
        else:
            if(pre == 1):
                errCount[2] += 1
            else:
                errCount[1] += 1

    print(modelPath.split('.')[1].split('-')[0],errCount[0],errCount[1],errCount[2],errCount[3])
    

01 249 1 3992 0
02 227 1469 2524 22
03 14 3984 9 235
04 242 275 3718 7
05 119 3899 94 130
06 183 3959 34 66
07 247 3050 943 2
08 215 2954 1039 34
09 222 3941 52 27
10 247 2627 1366 2
11 232 3876 117 17
12 40 3988 5 209
13 134 3986 7 115
14 241 3910 83 8
15 248 3566 427 1
16 249 22 3971 0
17 223 3911 82 26
18 248 3248 745 1
19 248 2659 1334 1
